In [23]:
import numpy as np
import cvxpy as cp

Toy Example

In [ ]:
X

### Examples of cvxpy and cvxpylayers

In [2]:
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

In [16]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
objective = cp.Minimize(c.T@x + cp.sum_squares(x) * 1e-2)
constraints = [
    A @ x <= b,
    cp.sum_squares(A @ x) <= np.sum(b ** 2) / 2
]
prob = cp.Problem(objective, constraints)
prob.solve()

-13.388855525388324

In [12]:
# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)


The optimal value is -13.48916108876133
A solution x is
[-1.21344369 -0.22703139 -0.82121289 -0.4959934   0.12467037 -0.46052435
  1.49070295  0.42483225  1.20058554  1.87999998]
A dual solution is
[2.12092392e-09 1.19413482e+00 2.07031246e+00 1.04310871e-08
 4.24186157e-09 1.45569757e+00 5.12293992e-09 1.03816048e-08
 3.90312958e-09 3.30334738e-09 6.30526169e-01 2.66488454e-07
 2.87627667e-01 2.72986790e-01 2.40614618e-09]


In [19]:
n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x + cp.sum_squares(x) <= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

In [21]:
import cvxpy as cp
import torch
from cvxpylayers.torch import CvxpyLayer

n, m = 2, 3
x = cp.Variable(n)
A = cp.Parameter((m, n))
b = cp.Parameter(m)
constraints = [x >= 0]
objective = cp.Minimize(0.5 * cp.pnorm(A @ x - b, p=1))
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables=[x])
A_tch = torch.randn(m, n, requires_grad=True)
b_tch = torch.randn(m, requires_grad=True)

# solve the problem
solution, = cvxpylayer(A_tch, b_tch)

# compute the gradient of the sum of the solution with respect to A, b
solution.sum().backward()